# Data Pre-processing

In [1]:
DATA_PATH = "../output/csv"
OUTPUT_PATH = "../output/data"

SAMPLES = 2000
TRAIN_RATIO = 0.70
VAL_RATIO = 0.20
TEST_RATIO = 0.10
RANDOM_STATE = 42

In [2]:
from pathlib import Path
from pandas import read_csv
from numpy import concat
from logging import basicConfig, INFO
from sys import float_info
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from joblib import dump

basicConfig(level=INFO)

from dataset_type import Dataset

In [3]:
DATA_PATH = Path(DATA_PATH).resolve()
OUTPUT_PATH = Path(OUTPUT_PATH).resolve()

OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

assert (1.0 - (TRAIN_RATIO + VAL_RATIO + TEST_RATIO)) < float_info.epsilon

In [4]:
def iter_files(dir_: Path):
    for file in filter(lambda x: x.is_file() and x.is_file(), dir_.iterdir()):
        yield (file.name.split(".")[0], file)

In [5]:
datasets: dict[str, Dataset] = {}

for station, file in iter_files(Path(DATA_PATH)):
    df = read_csv(file, parse_dates=["Datetime"])
    df = df.tail(SAMPLES).reset_index(drop=True)

    X = df[["pH", "EC", "Temp"]].values
    y = df["DO"].values.reshape(-1, 1)

    assert len(df) == SAMPLES

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=TRAIN_RATIO, random_state=RANDOM_STATE)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=TEST_RATIO / (TEST_RATIO + VAL_RATIO), random_state=RANDOM_STATE)

    X_scaler = MinMaxScaler().fit(concat((X_train, X_val)))
    y_scaler = MinMaxScaler().fit(concat((y_train, y_val)))

    X_train_mms = X_scaler.transform(X_train)
    y_train_mms = y_scaler.transform(y_train)

    X_val_mms = X_scaler.transform(X_val)    
    y_val_mms = y_scaler.transform(y_val)

    X_test_mms = X_scaler.transform(X_test)
    y_test_mms = y_scaler.transform(y_test)

    print(f"Station: {station}")

    datasets[station] = Dataset(
        df[["pH", "EC", "Temp"]], df["DO"],
        X_train, X_val, X_test, y_train, y_val, y_test,
        X_scaler, y_scaler,
        X_train_mms, X_val_mms, X_test_mms, y_train_mms, y_val_mms, y_test_mms
    )

Station: ป่าสัก3สถานี นครหลวง 2558-2563
Station: ป่าสัก3สถานี แก่งคอย 2558-2563
Station: ป่าสัก3สถานี เสาไห้ 2558-2563
Station: ยม สุโขทัย 2558-2563
Station: ยม สามง่าม 2558-2563
Station: ยม โพทะเล 2558-2563
Station: วัง เกาะคา 2558-2563
Station: แม่น้ำท่าจีน กระทุ่มแบน 2558-2563
Station: แม่น้ำท่าจีน บางเลน 2558-2563
Station: แม่น้ำท่าจีน สองพี่น้อง 2558-2563
Station: แม่น้ำท่าจีน นครชัยศรี 2558-2563
Station: แม่น้ำท่าจีน หันคา 2558-2563
Station: แม่น้ำท่าจีน สุพรรณบุรี 2558-2563
Station: แม่น้ำท่าจีน สามชุก 2558-2563
Station: ปิง เชียงใหม่ 2558-2563
Station: ปิง กำแพงเพชร 2558-2563
Station: น่าน อุตรดิตถ์ 2558-2563
Station: น่าน น่าน 2558-2563
Station: น่าน พิษณุโลก 2558-2563


In [6]:
_ = dump(datasets, OUTPUT_PATH / "dataset.joblib")